In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme()

In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

import statsmodels.api as sm


In [3]:
general = pd.read_csv('src/CC GENERAL.csv')

In [4]:
general.set_index('CUST_ID', drop=True, inplace=True)

In [5]:
mean_imputer = SimpleImputer(missing_values=np.nan, strategy='mean')

In [6]:
general_with_mean = general.copy()

In [7]:
general_with_mean['MINIMUM_PAYMENTS'] = mean_imputer.fit_transform(general_with_mean[['MINIMUM_PAYMENTS']])

In [8]:
general_with_mean = general_with_mean.dropna(axis = 0)

In [9]:
general_without_outlies = general_with_mean.copy()

In [10]:
def detect_outliers(col):
    Q1, Q3 = col.quantile([0.25,0.75])
    ric = Q3 - Q1
    li = Q1 - (1.5 * ric)
    ls = Q3 + (1.5 * ric)
    return li, ls

In [11]:
columns_list = general_without_outlies.columns
for i in columns_list:
    li, ls = detect_outliers(general_without_outlies[i])
    general_without_outlies[i] = np.where(general_without_outlies[i] > ls, ls,general_without_outlies[i])
    general_without_outlies[i] = np.where(general_without_outlies[i] < li, li,general_without_outlies[i])

In [12]:
selected_general_scaled = general_without_outlies.copy()

selected_general_scaled.iloc[:, :] = StandardScaler().fit_transform(general_without_outlies)

In [13]:
selected_general_scaled

,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
CUST_ID,,,,,,,,,,,,,,,,,
C10001,-0.877887,-1.022232,-0.729778,-0.692329,-0.505286,-0.673526,-0.806649,-0.722810,-0.707409,-0.730019,-0.697305,-0.738061,-1.010503,-0.899957,-0.794165,-0.629326,0.0
C10002,1.178347,-0.202948,-0.838253,-0.692329,-0.746088,2.115243,-1.221928,-0.722810,-0.917090,0.717780,0.473022,-0.891422,0.762675,2.165124,0.836651,0.973858,0.0
C10003,0.718318,0.616337,0.040879,0.817575,-0.746088,-0.673526,1.269742,2.210769,-0.917090,-0.730019,-0.697305,0.028748,0.910439,-0.569767,0.058585,-0.629326,0.0
C10004,0.179488,-1.887032,0.866182,2.128745,-0.746088,-0.467436,-1.014290,-0.396858,-0.917090,-0.247421,-0.404724,-0.814741,0.910439,-1.058507,0.472782,-0.629326,0.0
C10005,-0.372660,0.616337,-0.820060,-0.661083,-0.746088,-0.673526,-1.014290,-0.396858,-0.917090,-0.730019,-0.697305,-0.814741,-0.951397,-0.525559,-0.610107,-0.629326,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C19186,-0.885957,0.616337,-0.507235,-0.692329,-0.011263,-0.673526,1.269742,-0.722810,1.179720,-0.730019,-0.697305,-0.431337,-1.010503,-0.802697,-0.952597,1.947220,0.0
C19187,-0.892012,0.616337,-0.497138,-0.692329,0.011151,-0.673526,1.269742,-0.722810,1.179720,-0.730019,-0.697305,-0.431337,-1.010503,-0.841770,0.472782,-0.629326,0.0
C19188,-0.889270,-0.885689,-0.674063,-0.692329,-0.381604,-0.673526,0.854463,-0.722810,0.760359,-0.730019,-0.697305,-0.508018,-1.010503,-0.994655,-0.893975,1.174258,0.0


In [14]:
Xs = selected_general_scaled.drop(['CREDIT_LIMIT'], axis=1)

y = selected_general_scaled['CREDIT_LIMIT']

In [15]:
reg = LinearRegression()

In [16]:
model = reg.fit(Xs, y)

In [98]:
print("O modelo é: Limite de Crédito = {:.5} \
    + {:.5}*TV + {:.5}*radio + {:.5}*newspaper".format(reg.intercept_[0], reg.coef_[0][0], reg.coef_[0][1], reg.coef_[0][2]))

IndexError: invalid index to scalar variable.

In [21]:
X2 = sm.add_constant(Xs)

In [22]:
results = sm.OLS(y, X2).fit()

In [23]:
results.summary()

C:\Users\DESKTOP\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\regression\linear_model.py:1917: RuntimeWarning: divide by zero encountered in double_scalars
  return np.sqrt(eigvals[0]/eigvals[-1])


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           CREDIT_LIMIT   R-squared:                       0.419
Model:                            OLS   Adj. R-squared:                  0.418
Method:                 Least Squares   F-statistic:                     429.5
Date:                Tue, 02 Nov 2021   Prob (F-statistic):               0.00
Time:                        11:41:38   Log-Likelihood:                -10268.
No. Observations:                8949   AIC:                         2.057e+04
Df Residuals:                    8933   BIC:                         2.068e+04
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
const                            -9.263e-17      0.008  -1.15e-14      1.000      -0.016       0.016
BALANCE                              0.7125      0.017     43.076      0.000       0.680       0.745
BALANCE_FREQUENCY                   -0.1333      0.009    -14.038      0.000      -0.152      -0.115
PURCHASES                            0.0122      0.032      0.377      0.706      -0.051       0.076
ONEOFF_PURCHASES                     0.0415      0.026      1.569      0.117      -0.010       0.093
INSTALLMENTS_PURCHASES               0.0847      0.022      3.877      0.000       0.042       0.127
CASH_ADVANCE                         0.1099      0.015      7.230      0.000       0.080       0.140
PURCHASES_FREQUENCY                 -0.0231      0.026     -0.874      0.382      -0.075       0.029
ONEOFF_PURCHASES_FREQUENCY           0.1564      0.021      7.546      0.000       0.116       0.197
PURCHASES_INSTALLMENTS_FREQUENCY     0.0114      0.028      0.404      0.686      -0.044       0.067
CASH_ADVANCE_FREQUENCY              -0.1318      0.023     -5.824      0.000      -0.176      -0.087
CASH_ADVANCE_TRX                    -0.0489      0.024     -2.054      0.040      -0.096      -0.002
PURCHASES_TRX                       -0.0382      0.020     -1.870      0.062      -0.078       0.002
PAYMENTS                             0.1536      0.011     13.634      0.000       0.131       0.176
MINIMUM_PAYMENTS                    -0.1869      0.014    -13.308      0.000      -0.214      -0.159
PRC_FULL_PAYMENT                     0.1530      0.010     15.315      0.000       0.133       0.173
TENURE                                    0          0        nan        nan           0           0
==============================================================================
Omnibus:                     1858.187   Durbin-Watson:                   1.906
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3716.357
Skew:                           1.245   Prob(JB):                         0.00
Kurtosis:                       4.942   Cond. No.                          inf
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is      0. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [36]:
Xs = selected_general_scaled.drop(['CREDIT_LIMIT', 'PURCHASES', 'PURCHASES_INSTALLMENTS_FREQUENCY', 'TENURE', 'PURCHASES_FREQUENCY'], axis=1)

In [37]:
X2 = sm.add_constant(Xs)

In [38]:
results = sm.OLS(y, X2).fit()

In [39]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           CREDIT_LIMIT   R-squared:                       0.419
Model:                            OLS   Adj. R-squared:                  0.418
Method:                 Least Squares   F-statistic:                     536.9
Date:                Tue, 02 Nov 2021   Prob (F-statistic):               0.00
Time:                        11:56:58   Log-Likelihood:                -10269.
No. Observations:                8949   AIC:                         2.056e+04
Df Residuals:                    8936   BIC:                         2.066e+04
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
const                      -9.263e-17      0.008  -1.15e-14      1.000      -0.016       0.016
BALANCE                        0.7142      0.016     43.394      0.000       0.682       0.746
BALANCE_FREQUENCY             -0.1347      0.009    -14.392      0.000      -0.153      -0.116
ONEOFF_PURCHASES               0.0519      0.015      3.407      0.001       0.022       0.082
INSTALLMENTS_PURCHASES         0.0886      0.013      6.584      0.000       0.062       0.115
CASH_ADVANCE                   0.1093      0.015      7.207      0.000       0.080       0.139
ONEOFF_PURCHASES_FREQUENCY     0.1478      0.016      9.337      0.000       0.117       0.179
CASH_ADVANCE_FREQUENCY        -0.1304      0.023     -5.774      0.000      -0.175      -0.086
CASH_ADVANCE_TRX              -0.0489      0.024     -2.051      0.040      -0.096      -0.002
PURCHASES_TRX                 -0.0425      0.017     -2.518      0.012      -0.076      -0.009
PAYMENTS                       0.1562      0.011     14.392      0.000       0.135       0.177
MINIMUM_PAYMENTS              -0.1880      0.014    -13.461      0.000      -0.215      -0.161
PRC_FULL_PAYMENT               0.1512      0.010     15.456      0.000       0.132       0.170
==============================================================================
Omnibus:                     1857.455   Durbin-Watson:                   1.906
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3713.498
Skew:                           1.244   Prob(JB):                         0.00
Kurtosis:                       4.941   Cond. No.                         7.74
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [126]:
pred = model.predict(Xs)

In [132]:
r2 = r2_score(y, pred)

In [133]:
VIF = 1 - r2

In [134]:
VIF

0.5445072402547994